In [1]:
# Instalacion Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
#  Establecer variables de entorno:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
!ls

sample_data  spark-3.1.1-bin-hadoop3.2	spark-3.1.1-bin-hadoop3.2.tgz


In [4]:
# crear sesión 
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Propiedad utilizada para formatear mejor las tablas de salida
spark

In [10]:
# crear punto de montage Google -drive 
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/EjemplosPY/archivos/'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
#  Exploración del conjunto de datos
# Cargar datos de csv a un dataframe. 
# df = spark.read.csv('cars.csv', header=True, sep=";")
df = spark.read.parquet(path+'Habeas_Data_y_Exclusiones__c.parquet')
df.show(5)

+------------------+---------+-----------+------------------+-------------------+------------------+-------------------+------------------+-------------------+--------------------+----------------+------------------+---------+--------------------+--------------------+------------------+--------------------+--------------------+
|                Id|IsDeleted|       Name|      RecordTypeId|        CreatedDate|       CreatedById|   LastModifiedDate|  LastModifiedById|     SystemModstamp|ConnectionReceivedId|ConnectionSentId|       Contacto__c|Activo__c|      Comentarios__c|Fecha_de_Registro__c|         LongID__c|             Tipo__c|   Valor_Numerico__c|
+------------------+---------+-----------+------------------+-------------------+------------------+-------------------+------------------+-------------------+--------------------+----------------+------------------+---------+--------------------+--------------------+------------------+--------------------+--------------------+
|a2B8W0000

In [7]:
# Registrar tabla temporal
df.createOrReplaceTempView("temp")
# Seleccionar todos los datos de la tabla temporal
spark.sql("select * from temp limit 5").show()
# Seleccionar el recuento de datos de la tabla
spark.sql("select count(*) as total_count from temp").show()

+------------------+---------+-----------+------------------+-------------------+------------------+-------------------+------------------+-------------------+--------------------+----------------+------------------+---------+--------------------+--------------------+------------------+--------------------+--------------------+
|                Id|IsDeleted|       Name|      RecordTypeId|        CreatedDate|       CreatedById|   LastModifiedDate|  LastModifiedById|     SystemModstamp|ConnectionReceivedId|ConnectionSentId|       Contacto__c|Activo__c|      Comentarios__c|Fecha_de_Registro__c|         LongID__c|             Tipo__c|   Valor_Numerico__c|
+------------------+---------+-----------+------------------+-------------------+------------------+-------------------+------------------+-------------------+--------------------+----------------+------------------+---------+--------------------+--------------------+------------------+--------------------+--------------------+
|a2B8W0000

In [16]:
dfl = spark.read.parquet(path+'INFO_LABORAL.parquet')
dfl.show(5)


+------------------+---------------+----------------+--------------------+--------------------+--------------------+----------+------------------+
|       ID_CONTACTO|          CARGO|NIVEL_JERARQUICO|  SECTOR_ECONOMICO_1|  SECTOR_ECONOMICO_2|           EMPLEADOR|       NIT|      Empleador__c|
+------------------+---------------+----------------+--------------------+--------------------+--------------------+----------+------------------+
|0031I00000AiaJBQAZ|           null|            null|                null|                null|                null|      null|              null|
|0031I00000AiaJCQAZ|           null|            null|                null|                null|                null|      null|              null|
|0031I00000AiaJDQAZ|           null|            null|                null|                null|                null|      null|              null|
|0031I00000AibtmQAB|GERENTE GENERAL|       Directivo|Actividades de ag...|Actividades de se...|ATRAPALO COLOMBIA...|90

In [17]:
dfc = spark.read.parquet(path+'INFO_CONTACTO.parquet')
dfc.show(5)

+------------------+------------+----------------+----+--------------------+---------+--------+-----------+------------+--------------------+-----------+-----------+----------+-------------+------------------+--------------+----------+-------------------------+-----------------+-----------+-----------------------------------+-------------------------------+-----------------------------------+-------------------+-----------------------+-------+-----------+---------------------+----------------+------------+------------+---------------+------------------+-------------+----------------------+-------------------------------------+--------------------------------------+--------+
|       ID_CONTACTO|ESTADO_CIVIL|FECHA_NACIMIENTO|EDAD|          GENERACION|   GENERO|    PAIS|     CIUDAD|DEPARTAMENTO|           DIRECCION|ES_GRADUADO|ES_EXALUMNO|ES_ANTIGUO|ES_ESTUDIANTE|ES_MIEMBRO_CONCEJO|ES_PADRE_MADRE|ES_DONANTE|ES_RESPONSABLE_FINANCIERO|ES_ADMINISTRATIVO|ES_PROFESOR|ES_ANTIGUO_MIEMBRO_CONSEJO

In [18]:
dfc.join(dfl, dfl.ID_CONTACTO == dfc.ID_CONTACTO , 'inner').select(dfc['ID_CONTACTO'],dfc['FECHA_NACIMIENTO'],dfl['CARGO']).show(truncate=False) 

+------------------+----------------+---------------+
|ID_CONTACTO       |FECHA_NACIMIENTO|CARGO          |
+------------------+----------------+---------------+
|0031I00000AiaJBQAZ|null            |null           |
|0031I00000AiaJCQAZ|null            |null           |
|0031I00000AiaJDQAZ|null            |null           |
|0031I00000AibtmQAB|null            |GERENTE GENERAL|
|0031I00000AibtnQAB|1988-11-16      |null           |
|0031I00000AibtoQAB|null            |null           |
|0031I00000AibtpQAB|null            |GERENTE GENERAL|
|0031I00000AibuaQAB|null            |null           |
|0031I00000AibubQAB|1986-03-01      |AFILIACIONES   |
|0031I00000AibudQAB|1980-11-14      |GERENTE        |
|0031I00000AibwaQAB|null            |null           |
|0031I00000AibwbQAB|null            |null           |
|0031I00000AibwcQAB|null            |null           |
|0031I00000AibwdQAB|null            |null           |
|0031I00000AibweQAB|null            |null           |
|0031I00000AibwEQAR|null    

In [19]:
dfc.select('ESTADO_CIVIL').distinct().show()

+---------------+
|   ESTADO_CIVIL|
+---------------+
|           null|
|     C - CASADO|
|U - UNION LIBRE|
|    S - SOLTERO|
| D - DIVORCIADO|
|   P - SEPARADO|
|      V - VIUDO|
+---------------+



In [20]:
dfc.groupBy("ESTADO_CIVIL").count().orderBy('count', ascending=False).show(10)

+---------------+------+
|   ESTADO_CIVIL| count|
+---------------+------+
|           null|196615|
|    S - SOLTERO| 94381|
|     C - CASADO| 38311|
|U - UNION LIBRE|  1839|
|   P - SEPARADO|   765|
| D - DIVORCIADO|   758|
|      V - VIUDO|   164|
+---------------+------+

